## Imports

In [1]:
import logging
import os
import random
import sys
from collections import deque
from operator import itemgetter

import gym_donkeycar
import gymnasium as gym
import imageio
import ipywidgets as widgets
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from IPython.display import display
from ipywidgets import HBox, VBox
from matplotlib import pyplot as plt
from PIL import Image
from ruamel.yaml import YAML
from scipy.ndimage import gaussian_filter1d
from scipy.stats import norm
from tensorboard import notebook
from tensorboard.backend.event_processing.event_accumulator import \
    EventAccumulator
from torch import distributions as dist
from torch.distributions import Categorical, Normal
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
from tqdm import tqdm

# suppress warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="gymnasium")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ["IMAGEIO_IGNORE_WARNINGS"] = "True"

import gym.spaces as gym_spaces
import gymnasium as gym  # overwrite OpenAI gym
import stable_baselines3 as sb3
from gym_donkeycar.envs.donkey_env import DonkeyEnv
from gymnasium import spaces
from gymnasium.spaces import Box
from gymnasium.experimental.wrappers import RescaleActionV0
from stable_baselines3 import A2C, PPO, SAC
from stable_baselines3.common import env_checker
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.results_plotter import load_results, ts2xy
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv

from src.actor_critic import ContinuousActorCritic
from src.blocks import CategoricalStraightThrough, ConvBlock
from src.categorical_vae import CategoricalVAE
from src.imagination_env import ImaginationEnv
from src.mlp import MLP
from src.preprocessing import grayscale_transform as transform
from src.replay_buffer import ReplayBuffer
from src.rssm import RSSM
from src.utils import (load_config, make_env, save_image_and_reconstruction,
                       to_np)
from src.vae import VAE

torch.cuda.empty_cache()
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Load the config
config = load_config()
for key in config:
    locals()[key] = config[key]

## Create the environment

In [60]:
env = make_env()

Making a toy env.
Making 1 vectorized envs.
Adding a Gymnasium RecordEpisodeStatistics wrapper.
Adding a TimeLimit wrapper with 1000 max episode steps.
Adding an AutoReset wrapper.
Adding a RescaleActionV0 wrapper to have an effective action space [-1,1].
Note: Clip actions at 0.5 => The agent can take agents from:
Low: [-0.5 -0.5 -0.5] to High: [0.5 0.5 0.5]


In [61]:
vae = VAE().to(device)
vae.optim = optim.Adam(vae.parameters(), lr=1e-4, weight_decay=1e-6)

Initializing encoder:
- adding ConvBlock((3, 64))                   ==> output shape: (64, 32, 32) ==> prod: 65536
- adding ConvBlock((64, 128))                   ==> output shape: (128, 16, 16) ==> prod: 32768
- adding ConvBlock((128, 256))                   ==> output shape: (256, 8, 8) ==> prod: 16384
- adding ConvBlock((256, 512))                   ==> output shape: (512, 4, 4) ==> prod: 8192
- adding ConvBlock((512, 256))                   ==> output shape: (256, 2, 2) ==> prod: 1024
- adding Flatten()
- adding Linear() for Mu: 1024 and Logvar: 1024

Initializing decoder:
- adding Reshape: (*,1024) => (*,256,2,2)
- adding transpose ConvBlock(256, 256)                   ==> output shape: (256, 4, 4) ==> prod: 4096
- adding transpose ConvBlock(256, 512)                   ==> output shape: (512, 8, 8) ==> prod: 32768
- adding transpose ConvBlock(512, 256)                   ==> output shape: (256, 16, 16) ==> prod: 65536
- adding transpose ConvBlock(256, 128)                   ==> out

In [ ]:
env.action_space.low

In [ ]:
z.shape

In [ ]:
agent.get_action(z)

In [ ]:
# New training loop with batches

losses = {
    "vae_loss": [],
    "reconstruction_loss": [],
    "KLD_loss": [],
}

for sample_phase in tqdm(range(n_updates)):
    
    batch_observations = []
    batch_reconstructions = []
    batch_mu = []
    batch_logvar = []
    
    if sample_phase == 0:
        obs, info = env.reset(seed=42)

    for step in range(n_steps_per_update):
        if len(obs.shape) == 4:
            obs = obs[0]

        # Get the observation and encode it
        obs = transform(obs)
        z, reconstruction, mu, logvar = vae(obs)
        
        if sample_phase % 100 == 0:
            save_image_and_reconstruction(obs, reconstruction, sample_phase)
        
        # Add the observation, reconstruction, mu, and logvar to the respective batches
        batch_observations.append(obs)
        batch_reconstructions.append(reconstruction)
        batch_mu.append(mu)
        batch_logvar.append(logvar)

        # Get an action and take an environment step
        action = np.random.rand(A)
        obs, reward, terminated, truncated, info = env.step(action)

        
    # Convert the batch tensors to tensors and update vae
    batch_observations = torch.stack(batch_observations) # [128, 3, 64, 64]
    batch_reconstructions = torch.stack(batch_reconstructions) # [128, 3, 64, 64]
    batch_mu = torch.stack(batch_mu) # [128, latent_dim]
    batch_logvar = torch.stack(batch_logvar) # [128, latent_dim]
    
    vae_loss, reconstruction_loss, KLD_loss = vae.get_loss(batch_observations, batch_reconstructions, batch_mu, batch_logvar)
    vae.optim.zero_grad()
    vae_loss.backward()
    vae.optim.step()

    # Log the losses
    losses["vae_loss"].append(to_np(vae_loss))
    losses["reconstruction_loss"].append(to_np(reconstruction_loss))
    losses["KLD_loss"].append(to_np(KLD_loss))


In [ ]:
# max_episode_steps, 

In [ ]:
plt.plot(losses["vae_loss"])

In [ ]:
plt.plot(losses["reconstruction_loss"])

In [ ]:
plt.plot(losses["KLD_loss"])

In [ ]:
obs, info = env.reset(seed=42)

if len(obs.shape) == 4:
    obs = obs[0]

obs = transform(obs)

plt.imshow(to_np(torch.permute(obs, (1,2,0))))

In [ ]:
reconstruction, mu, logvar = vae(torch.randn(obs.shape).to(device))
plt.imshow(to_np(torch.permute(reconstruction, (1,2,0))))